In [64]:
# ====== notifier.py（直接貼到你的腳本頂端就好）======
import os, sys, time, platform, subprocess
from pathlib import Path

def _beep_fallback(times=2, freq=1000, dur_ms=500):
    """盡力嗶一下（Windows winsound / 終端 bell / macOS say）。"""
    try:
        if platform.system() == "Windows":
            import winsound
            for _ in range(times):
                winsound.Beep(freq, dur_ms)
                time.sleep(0.15)
            return True
        elif platform.system() == "Darwin":
            # macOS：系統語音 or terminal bell
            os.system('say "job finished"')  # 若你不想講話可改成 print('\a')
            return True
        else:
            # Linux/其他：終端 bell
            sys.stdout.write('\a' * times); sys.stdout.flush()
            return True
    except Exception:
        pass
    return False

def _toast_windows(title, msg, duration=5):
    """Windows Toast（需要 pip install win10toast）。"""
    try:
        from win10toast import ToastNotifier
        ToastNotifier().show_toast(title, msg, duration=duration, threaded=True)
        return True
    except Exception:
        return False

def _toast_plyer(title, msg):
    """通用桌面通知（pip install plyer）。"""
    try:
        from plyer import notification
        notification.notify(title=title, message=msg, timeout=5)
        return True
    except Exception:
        return False

def _toast_linux(title, msg):
    """Linux notify-send（系統通常自帶）。"""
    try:
        subprocess.Popen(["notify-send", title, msg])
        return True
    except Exception:
        return False

def _popup_windows(title, msg):
    """Windows 原生彈窗（不需安裝套件）。"""
    try:
        import ctypes
        ctypes.windll.user32.MessageBoxW(0, msg, title, 0x40)  # MB_ICONINFORMATION
        return True
    except Exception:
        return False

def notify_done(msg="任務完成 ✅", title="OPM 批次任務"):
    """
    完成時叫你：優先桌面通知→彈窗→嗶聲→終端提示。
    放在任務結尾，或 except/finally 中。
    """
    ok = False
    system = platform.system()

    # 桌面通知（優先）
    if system == "Windows":
        ok = _toast_windows(title, msg) or _toast_plyer(title, msg)
    elif system == "Darwin":
        # macOS: 優先 plyer，其次 AppleScript
        ok = _toast_plyer(title, msg)
        if not ok:
            try:
                script = f'display notification "{msg}" with title "{title}"'
                subprocess.Popen(["osascript", "-e", script])
                ok = True
            except Exception:
                ok = False
    else:
        # Linux
        ok = _toast_plyer(title, msg) or _toast_linux(title, msg)

    # 退而求其次：Windows 原生彈窗
    if not ok and system == "Windows":
        ok = _popup_windows(title, msg)

    # 再退：嗶聲/語音
    if not ok:
        ok = _beep_fallback(times=2)

    # 最後保底輸出
    if not ok:
        print(f"\n=== {title} ===\n{msg}\n")

def run_with_notify(func, *args, title="OPM 批次任務", **kwargs):
    """
    用法：run_with_notify(process_root, root_dir)
    成功與失敗都會通知；失敗會把例外訊息也帶出來。
    """
    try:
        result = func(*args, **kwargs)
        notify_done("已執行完成 ✅", title=title)
        return result
    except Exception as e:
        notify_done(f"執行失敗 ❌：{e}", title=title)
        raise

# batch_opm_summary.py
# -*- coding: utf-8 -*-
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import json, re
# 依賴你的工具：請確認 opmtool 在可匯入的路徑
from opmtool import (
    dispersion_lorentz_fit, voigt_fit, lorentz_fit, gauss_fit,
    one_cycle_cut, noise_psd, noise_psd_lowband
)
import warnings
# 後端用 Agg，避免彈視窗 & 紅字
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# =========================
# 🔧 全域設定
# =========================
# ---- 輸出控制 ----
XLSX_SHEET_MODE = "all_only"   # "all_only" | "per_session"
PRINT_SUMMARY_TO_STDOUT = True # 跑完直接把 ALL 總表印在終端機
STDOUT_MAX_ROWS = 200          # 避免超長洗版；想全印可設 None

# ---- 圖像輸出設定 ----
PLOT_ENABLED = True          # ← 開/關 作圖存檔
PLOT_DPI     = 140
PLOT_FMT     = "png"
PLOT_STYLE   = "default"     # 你喜歡的 matplotlib style，或 "seaborn-v0_8"
PLOT_MAXPTS  = 4000          # 畫點太多會很慢，超過就等距抽樣繪點
plt.style.use(PLOT_STYLE)
# ===== 實驗模式(有預設資料夾名稱則會自動尋找) =====
# "current" = 固定光強改波長（資料夾 128, 132 表「雷射電流刻度」）
# "power"   = 固定頻率掃光強（資料夾 160, 180 表「AOM/衰減器控制電壓」等）
EXPERIMENT_MODE = "current"   # or "current"

# 若是 "current" 模式，要把資料夾數字加上校正（例如 128 -> 128.4 mA）
CURRENT_OFFSET_MA = 0.4

# 若是 "power" 模式：把資料夾數字視為控制電壓（單位自訂）
POWER_UNIT   = "mV"        # 你實際的單位（mV or V）
POWER_SCALE  = 1.0         # 需要就換算 (顯示值 = 原值*POWER_SCALE + POWER_OFFSET)
POWER_OFFSET = 0.0

# 掃場切半、B 映射
B_RANGE_NT     = (-32.12, 32.12)#如果沒有實驗記錄
LINE_SEGMENT   = 'half'       # 'half' | 'full'
LINE_DIRECTION = 'auto'       # 'auto' | 'rising' | 'falling'

# 斜率擬合：中心自動估（零交越），視窗寬度 = 全域 B 範圍的百分比（半寬）
DISP_CENTER   = 0.0           # 僅在 fallback 時當預設
WIDTHRATIO    = 10.0           # 5% → 半寬 = 0.05 * (max(B)-min(B))
DISP_P0_AMPL = 0.01

# 雜訊計算
NOISE_MODE                = "lowband"  # "lowband" | "plain"
NOISE_BAND                = (3.0, 80.0)
NOISE_LOCK_DF_HZ          = 0.20
NOISE_DETREND             = "linear"   # "linear" | "constant" | None
NOISE_ENABLE_PREPROCESS   = True
NOISE_WINDOW              = "hann"
NOISE_AVERAGE             = "median"

# 檔案選取
SCAN_CSV_GLOB   = "*.csv"
NOISE_SUBDIR    = "noise"
# 回合資料夾名稱樣式（e.g. '01', '02', '03'…）
SESSION_NAME_PATTERN = r"^\d{2,}$"     # 兩碼以上純數字就視為回合資料夾
CURRENT_OFFSET_MA    = 0.4             # 你的「128 → 128.4」這種常數偏移
EXCEL_PER_SESSION    = True            # True → 多分頁 .xlsx；False → 僅 CSV


# CSV 欄位選擇（大小寫不敏感）
# 可選： "slope", "sloper2", "noisepsd", "sensitivity",
#       "lorentzfwhm", "gaussianfwhm", "voigtfwhm", "voigtgamma", "voigtsigma"
SELECT_METRICS = [
    "slope", "sloper2", "noisepsd", "sensitivity",
    "lorentzfwhm",         
    "gaussianfwhm",       
    "voigtfwhm"
]

# 動態欄名（寫到 CSV）
if EXPERIMENT_MODE.lower() == "current":
    SWEEP_COL_NAME = "current_mA"
elif EXPERIMENT_MODE.lower() == "power":
    SWEEP_COL_NAME = f"power_ctrl_{POWER_UNIT}"
else:
    raise ValueError("EXPERIMENT_MODE must be 'current' or 'power'")
    
# 欄位對應（輸出名稱）
ALWAYS_COLUMNS = ["label", "folder_name", SWEEP_COL_NAME, "B_range_used_nT"]
_METRIC_COLNAMES = {
    "slope":        "slope_mV_per_nT",
    "sloper2":      "slope_R2",
    "noisepsd":     "noise_rms_uV_per_rtHz",
    "sensitivity":  "sensitivity_pT_per_rtHz",
    "lorentzfwhm":  "lorentz_FWHM_nT",
    "lorentzr2":    "lorentz_R2",
    "gaussianfwhm": "gaussian_FWHM_nT",
    "gaussianr2":   "gaussian_R2",
    "voigtfwhm":    "voigt_FWHM_nT",
    "voigtgamma":   "voigt_gamma_nT",
    "voigtsigma":   "voigt_sigma_nT",
    "voigtr2":      "voigt_R2", 
}

# 欄位位置對應（讀檔）
SCAN_USE_POSITION  = True
NOISE_USE_POSITION = True
SCAN_POS_MAP  = { "time": 0, "Ab": 1, "demod": 2, "tri": 3 }
NOISE_POS_MAP = { "time": 0, "Ab": 1, "demod": 2 }
USER_SCAN_COLNAMES  = { "time": "time", "Ab": "Ab", "demod": "demod", "tri": "tri" }
USER_NOISE_COLNAMES = { "time": "time", "Ab": "Ab", "demod": "demod" }

# 選單驗證 + 小工具
_SELECT = {s.lower() for s in SELECT_METRICS}
for s in _SELECT:
    if s not in _METRIC_COLNAMES:
        raise ValueError(f"未知的指標名稱: {s}")
def _want(name: str) -> bool:
    return name.lower() in _SELECT

def _extract_sweep_value(label: str) -> float:
    """依 EXPERIMENT_MODE 從資料夾名抽出 sweep 變數（電流或光強控制電壓）"""
    val = _extract_numeric_from_label(label)  # 先抓到資料夾中的數字
    if val is None:
        return np.nan
    mode = EXPERIMENT_MODE.lower()
    if mode == "current":
        return float(val) + float(CURRENT_OFFSET_MA)
    elif mode == "power":
        return float(val) * float(POWER_SCALE) + float(POWER_OFFSET)
    else:
        return np.nan

def _plot_and_save_dispersion(sub_dir, B, demod, yfit, slope, r2, lin_range=None, ctr=None, offset=None):
    try:
        B = np.asarray(B); demod = np.asarray(demod); yfit = np.asarray(yfit)
    
        plt.figure(figsize=(6.0,4.0))
        plt.plot(B, demod, '.', ms=2, color='k', label='Demod (window)')
        if np.all(np.isfinite(yfit)) and len(yfit)==len(B):
            plt.plot(B, yfit, '-', lw=2, color='r', label='Dispersion fit')
    
        # 線性範圍陰影 + 小訊號線性近似
        if lin_range and ctr is not None and offset is not None and np.all(np.isfinite(lin_range)):
            lo, hi = lin_range
            if lo < hi:
                plt.axvspan(lo, hi, color='tab:green', alpha=0.15, label='Linear range')
                Bb = np.linspace(lo, hi, 200)
                yl = offset + slope*(Bb - ctr)
                plt.plot(Bb, yl, 'g--', lw=1.8, label='Small-signal approx')
    
        plt.title(f"Dispersion fit  (R²={r2:.4f}, slope={slope*1e3:.3g} mV/nT)")
        plt.xlabel("Magnetic Field (nT)")
        plt.ylabel("Demodulated Signal (V)")
        plt.legend(loc='best', fontsize=9)
        plt.tight_layout()
    
        out_png = Path(sub_dir) / "fit_dispersion.png"
        plt.savefig(out_png, dpi=180)
        plt.close()
    except Exception as e:
        print(f"[WARN] 繪圖失敗（dispersion）：{sub_dir.name}: {e}")

def _plot_and_save_lineshapes(sub_dir: Path, B, Ab,  # data
                              want_lor, lf,          # dict 或 None
                              want_gau, gf,
                              want_voi, vf):
    try:
        x = np.asarray(B); y = np.asarray(Ab)
        if len(x) > PLOT_MAXPTS:
            idx = np.linspace(0, len(x)-1, PLOT_MAXPTS).astype(int)
            x, y = x[idx], y[idx]
        fig, ax = plt.subplots(figsize=(6,4))
        ax.plot(x, y, ".", ms=2, alpha=0.5, label="data")

        xx = np.linspace(np.nanmin(B), np.nanmax(B), 1200)
        if want_lor and lf:
            # f(B) = a/(1+(b*(B-center))^2)+c
            p = lf.get("Params", {})
            a, b, c0, ctr = p.get("amplitude"), p.get("b"), p.get("offset"), p.get("center")
            if all(v is not None for v in (a,b,c0,ctr)):
                yy = a/(1.0+(b*(xx-ctr))**2)+c0
                ax.plot(xx, yy, "--", lw=2, label=f"Lorentz (R²={_first_scalar(lf.get('R2')):.4f})")

        if want_gau and gf:
            # GaussianModel: amplitude, center, sigma, offset
            p = gf.get("Params", {})
            A, mu, sig, c0 = p.get("amplitude"), p.get("center"), p.get("sigma"), p.get("offset")
            if all(v is not None for v in (A,mu,sig,c0)):
                yy = (A/(np.sqrt(2*np.pi)*sig))*np.exp(-0.5*((xx-mu)/sig)**2) + c0
                ax.plot(xx, yy, "--", lw=2, label=f"Gaussian (R²={_first_scalar(gf.get('R2')):.4f})")

        if want_voi and vf:
            p = vf.get("Params", {})
            A, mu, sig, gam, c0 = p.get("amplitude"), p.get("center"), p.get("sigma"), p.get("gamma"), p.get("offset")
            if all(v is not None for v in (A,mu,sig,gam,c0)):
                # 用 area-normalized Voigt 再乘上面積 A，加上 offset
                from scipy.special import wofz
                def voigt_area(xx, sigma, gamma):
                    z = (xx + 1j*gamma) / (sigma*np.sqrt(2))
                    return np.real(wofz(z)) / (sigma*np.sqrt(2*np.pi))
                yy = A*voigt_area(xx-mu, max(1e-12,sig), max(1e-12,gam)) + c0
                ax.plot(xx, yy, "--", lw=2, label=f"Voigt (R²={_first_scalar(vf.get('R2')):.4f})")

        ax.set_xlabel("B (nT)"); ax.set_ylabel("Absorption (V)")
        ax.set_title("Lineshape fits")
        ax.legend(loc=1)
        out = sub_dir / f"fit_lineshape.{PLOT_FMT}"
        fig.tight_layout(); fig.savefig(out, dpi=PLOT_DPI); plt.close(fig)
    except Exception as e:
        print(f"[WARN] 繪圖失敗（lineshape）：{sub_dir.name}: {e}")

def _plot_and_save_psd(noise_dir: Path, noise_df_std: pd.DataFrame):
    try:
        if not noise_dir.exists():
            noise_dir.mkdir(parents=True, exist_ok=True)
        # 直接重算一次拿頻譜點來畫
        if NOISE_MODE == "lowband":
            pts, asd_mean, fs_used, info = noise_psd_lowband(
                noise_df_std,
                band=NOISE_BAND,
                window=NOISE_WINDOW,
                average=NOISE_AVERAGE,
                min_freq_resolution=NOISE_LOCK_DF_HZ,
                force_long_segments=False,
                detrend=NOISE_DETREND,
                snap_to_power_of_2=False,
                enable_auto_preprocess=NOISE_ENABLE_PREPROCESS
            )
        else:
            pts, asd_mean, fs_used, info = noise_psd(
                noise_df_std,
                band=NOISE_BAND,
                window=NOISE_WINDOW,
                average=NOISE_AVERAGE,
                min_freq_resolution=NOISE_LOCK_DF_HZ,
                force_long_segments=False,
                detrend=NOISE_DETREND,
                snap_to_power_of_2=False
            )
        f, Pxx = np.asarray(pts[:,0]), np.asarray(pts[:,1])
        asd = np.sqrt(np.maximum(Pxx, 0))
        fig, ax = plt.subplots(figsize=(6,4))
        ax.semilogy(f, asd, "-", lw=1.5, label="ASD (V/√Hz)")
        f1, f2 = NOISE_BAND
        ax.axvspan(f1, f2, alpha=0.15, label=f"band {f1:g}–{f2:g} Hz")
        ax.set_xlabel("Frequency (Hz)")
        ax.set_ylabel("ASD (V/√Hz)")
        ax.set_title(f"Noise PSD (mean={asd_mean*1e6:.3g} μV/√Hz)")
        ax.legend(loc="best")
        out = noise_dir / f"noise_psd.{PLOT_FMT}"
        fig.tight_layout(); fig.savefig(out, dpi=PLOT_DPI); plt.close(fig)
    except Exception as e:
        print(f"[WARN] 繪圖失敗（PSD）：{noise_dir.name}: {e}")

# 任何「選了擬合參數」→ 強制把對應 R² 一起輸出
def _force_include_r2_columns(selected: set[str]) -> set[str]:
    sel = set(selected)
    # slope：若選了 slope 就自動加 sloper2（你的規則 3）
    if "slope" in sel:
        sel.add("sloper2")
    # Voigt：只要有 voigt_* 就加 voigtR2
    if {"voigtfwhm","voigtgamma","voigtsigma"} & sel:
        sel.add("voigtr2")
    # Lorentz：若選 lorentzFWHM 就加 lorentzR2
    if "lorentzfwhm" in sel:
        sel.add("lorentzr2")
    # Gaussian：若選 gaussianFWHM 就加 gaussianR2
    if "gaussianfwhm" in sel:
        sel.add("gaussianr2")
    return sel

_SELECT = _force_include_r2_columns(_SELECT)

def _first_scalar(x, default=np.nan):
    """把 0-d/1-d 的 numpy/pandas/純數字，穩定轉成 float；其他回 default。"""
    try:
        if x is None:
            return default
        if hasattr(x, "values"):                  # pandas objects
            arr = np.asarray(x.values, dtype=float)
            return float(arr.ravel()[0])
        if isinstance(x, (list, tuple)):          # Python list/tuple
            return float(np.asarray(x, dtype=float).ravel()[0])
        arr = np.asarray(x)
        if arr.ndim >= 1:                         # numpy array
            return float(arr.ravel()[0])
        return float(arr)                         # 0-d numpy scalar or float
    except Exception:
        return default


def _load_experiment_rules(root_dir: Path):
    """
    從 root_dir/實驗記錄.txt 讀規則。
    支援範例行：
      160~300的掃磁範圍是+/-32.12 nT(22 mVpp)，200 mHz
      350~400的掃磁範圍是±64.24 nT(44 mVpp)，100 mHz
      500~550的掃磁範圍是+/-96.36 nT(66 mVpp)，70 mHz
    回傳 list[dict]: [{'lo':160,'hi':300,'Bpm':32.12,'mVpp':22,'fmHz':200}, ...]
    """
    p = root_dir / "實驗記錄.txt"
    if not p.exists():
        return []

    txt = p.read_text(encoding="utf-8-sig")
    rules = []
    # 允許「±」或「+/-」，允許中文/英文逗號，允許空白
    pattern = re.compile(
        r"""
        (?P<lo>\d+(?:\.\d+)?)\s*~\s*(?P<hi>\d+(?:\.\d+)?)       # 160~300
        .*?掃磁範圍.*?                                          # 任意字直到「掃磁範圍」
        (?:\+/-|±)\s*(?P<Bpm>\d+(?:\.\d+)?)\s*nT                # ±32.12 nT
        \s*\(\s*(?P<mVpp>\d+(?:\.\d+)?)\s*mVpp\s*\)            # (22 mVpp)
        [，,]\s*(?P<fmHz>\d+(?:\.\d+)?)\s*mHz                   # ，200 mHz
        """, re.VERBOSE | re.IGNORECASE
    )

    for line in txt.splitlines():
        m = pattern.search(line)
        if not m:
            continue
        lo  = float(m.group("lo"))
        hi  = float(m.group("hi"))
        Bpm = float(m.group("Bpm"))      # 正半幅（±B）
        mVpp= float(m.group("mVpp"))
        fmHz= float(m.group("fmHz"))
        # 清理 lo>hi 的奇怪寫法
        lo, hi = (lo, hi) if lo <= hi else (hi, lo)
        rules.append(dict(lo=lo, hi=hi, Bpm=Bpm, mVpp=mVpp, fmHz=fmHz))
    return rules

_num_re = re.compile(r"([-+]?\d+(?:\.\d+)?)")

def _extract_numeric_from_label(label: str) -> float | None:
    m = _num_re.search(label)
    return float(m.group(1)) if m else None

def _choose_rule_for_label(rules: list, label: str):
    """
    依資料夾名內數字落在哪個區間來選規則；多重命中取最窄區間。
    回傳 (B_range, meta) 或 (None, None)
    """
    val = _extract_numeric_from_label(label)
    if val is None or not rules:
        return None, None
    # 所有命中的規則
    hits = [r for r in rules if r["lo"] <= val <= r["hi"]]
    if not hits:
        return None, None
    # 取區間寬度最小者（避免重疊時歧義）
    best = min(hits, key=lambda r: (r["hi"] - r["lo"], -r["Bpm"]))
    Bpm  = best["Bpm"]
    B_range = (-Bpm, Bpm)
    meta = {"mVpp": best["mVpp"], "fmHz": best["fmHz"], "lo": best["lo"], "hi": best["hi"]}
    return B_range, meta

def _folder_has_scan_csv(folder: Path, root_for_pick: Path) -> bool:
    """資料夾內是否存在一個『掃描 CSV』可被 _pick_scan_csv() 挑到。"""
    try:
        return _pick_scan_csv(folder, root=root_for_pick) is not None
    except Exception:
        return False
def _detect_layout(root_dir: Path):
    """
    依『層級』自動判斷資料結構：
    - 若 root/ 自己就有掃描 CSV → ('single', [root], 'all_only')
    - 若 root/ 底下的第一層子資料夾就各自有掃描 CSV → ('flat', [root], 'all_only')
    - 若 root/ 底下第一層沒有，但其子層有掃描 CSV → ('session', [每個第一層子資料夾], 'per_session')
    - 其他情況 → 當成 single
    """
    # C. 根目錄自己是一筆
    if _folder_has_scan_csv(root_dir, root_for_pick=root_dir):
        return ('single', [root_dir], 'all_only')

    lvl1 = [p for p in sorted(root_dir.iterdir()) if p.is_dir() and p.name != NOISE_SUBDIR]
    if not lvl1:
        return ('single', [root_dir], 'all_only')

    # A. 扁平：第一層資料夾各自就是數據資料夾
    if any(_folder_has_scan_csv(d, root_for_pick=root_dir) for d in lvl1):
        return ('flat', [root_dir], 'all_only')

    # B. session：第一層不是數據，但其第二層才有數據
    session_dirs = []
    for s in lvl1:
        subdirs = [p for p in sorted(s.iterdir()) if p.is_dir() and p.name != NOISE_SUBDIR]
        if any(_folder_has_scan_csv(sd, root_for_pick=s) for sd in subdirs):
            session_dirs.append(s)

    if session_dirs:
        return ('session', session_dirs, 'per_session')

    # fallback
    return ('single', [root_dir], 'all_only')


def _is_summary_name(p: Path, root: Path) -> bool:
    """是否為主資料夾同名的彙整 CSV（需要避開避免誤讀）。"""
    summary_name = f"{root.name}.csv"
    return (p.name == summary_name)

def _pick_scan_csv(folder: Path, root: Path) -> Path | None:
    """挑選每個數據資料夾內的掃場 CSV；跳過 noise/ 與主彙整檔（root/<root>.csv 或 session/<session>.csv）。"""
    cands = []
    for csv in folder.glob(SCAN_CSV_GLOB):
        if csv.is_dir():
            continue
        # 跳過 noise
        if csv.parent.name == NOISE_SUBDIR:
            continue
        # 跳過『和 root 同名』的彙整 csv（避免誤讀）
        if csv.name == f"{root.name}.csv":
            continue
        cands.append(csv)
    return sorted(cands)[0] if cands else None

def _pick_noise_csv(noise_dir: Path) -> Path | None:
    """在 noise/ 子資料夾挑第一個 CSV。"""
    if not noise_dir.exists() or not noise_dir.is_dir():
        return None
    for csv in sorted(noise_dir.glob("*.csv")):
        if csv.is_file():
            return csv
    return None

def _standardize_scan_df(df_raw: pd.DataFrame) -> pd.DataFrame:
    """
    先依欄位順序擷取：
      time <- 第 SCAN_POS_MAP['time'] 欄
      Ab   <- 第 SCAN_POS_MAP['Ab']   欄
      demod<- 第 SCAN_POS_MAP['demod']欄
      tri  <- 第 SCAN_POS_MAP['tri']  欄
    若位置不足或失敗，才退回舊的「按欄名關鍵字」模式。
    最後依 USER_SCAN_COLNAMES 重新命名輸出欄名。
    """
    df_raw = df_raw.copy()
    out_names = USER_SCAN_COLNAMES

    # --- 位置優先 ---
    if SCAN_USE_POSITION:
        try:
            cols = {}
            for k, idx in SCAN_POS_MAP.items():
                if idx >= df_raw.shape[1]:
                    raise IndexError(f"掃描 CSV 欄數不足（需要第 {idx} 欄作 {k}）")
                cols[k] = pd.to_numeric(df_raw.iloc[:, idx], errors="coerce")
            df = pd.DataFrame(cols).dropna(how="any")
            # 重新命名為使用者指定
            df = df.rename(columns={k: out_names.get(k, k) for k in cols})
            return df
        except Exception:
            # 失敗就走名稱辨識
            pass

    # --- 名稱辨識（fallback） ---
    norm = {re.sub(r"\s+", "", c.lower()): c for c in df_raw.columns}
    def pick(key_part, fallback=None):
        for k, orig in norm.items():
            if key_part in k:
                return orig
        return fallback

    tcol = pick("time")
    Acol = pick("channela(", pick("channela"))
    Bcol = pick("channelb(", pick("channelb"))
    Ccol = pick("channelc(", pick("channelc"))
    if not (tcol and Acol and Bcol and Ccol):
        raise ValueError("掃描 CSV 缺少需要的欄位（需含 Time、Channel A/B/C 或提供足夠欄位順序）。")

    df = pd.DataFrame({
        out_names.get("time","time"):  pd.to_numeric(df_raw[tcol], errors="coerce"),
        out_names.get("Ab","Ab"):      pd.to_numeric(df_raw[Acol], errors="coerce"),
        out_names.get("demod","demod"):pd.to_numeric(df_raw[Bcol], errors="coerce"),
        out_names.get("tri","tri"):    pd.to_numeric(df_raw[Ccol], errors="coerce"),
    }).dropna(how="any")
    return df


def _standardize_noise_df(df_raw: pd.DataFrame) -> pd.DataFrame:
    """
    雜訊檔也先依欄位順序擷取（通常只有前三欄）：
      time <- NOISE_POS_MAP['time']
      Ab   <- NOISE_POS_MAP['Ab']     （可有可無，不用也無所謂）
      demod<- NOISE_POS_MAP['demod']  ← PSD 用這一欄
    若位置失敗再按名稱辨識。
    最後輸出三欄：time, sig1, sig2（sig2=demod），以符合 noise_psd(_lowband) 介面。
    """
    df_raw = df_raw.copy()
    out_names = USER_NOISE_COLNAMES

    # --- 位置優先 ---
    if NOISE_USE_POSITION:
        try:
            # time
            t_idx = NOISE_POS_MAP["time"]
            if t_idx >= df_raw.shape[1]:
                raise IndexError("noise CSV 欄數不足（time）")
            time = pd.to_numeric(df_raw.iloc[:, t_idx], errors="coerce")

            # demod
            d_idx = NOISE_POS_MAP["demod"]
            if d_idx >= df_raw.shape[1]:
                raise IndexError("noise CSV 欄數不足（demod）")
            demod = pd.to_numeric(df_raw.iloc[:, d_idx], errors="coerce")

            # Ab（可選）
            a_val = None
            if "Ab" in NOISE_POS_MAP and NOISE_POS_MAP["Ab"] < df_raw.shape[1]:
                a_val = pd.to_numeric(df_raw.iloc[:, NOISE_POS_MAP["Ab"]], errors="coerce")

            df = pd.DataFrame({
                out_names.get("time","time"): time,
                # PSD 函式需要三欄；sig1 可放 0 或 Ab，不影響我們用 sig2 算 PSD
                "sig1": (a_val if a_val is not None else 0.0),
                "sig2": demod,
            }).dropna(how="any")
            return df
        except Exception:
            pass

    # --- 名稱辨識（fallback） ---
    import re
    norm = {re.sub(r"\s+", "", c.lower()): c for c in df_raw.columns}
    def pick(key_part, fallback=None):
        for k, orig in norm.items():
            if key_part in k:
                return orig
        return fallback

    tcol = pick("time")
    Bcol = pick("channelb(", pick("channelb"))
    if not (tcol and Bcol):
        raise ValueError("noise CSV 需至少有 Time 與 Channel B，或提供足夠欄位順序。")

    df = pd.DataFrame({
        out_names.get("time","time"):  pd.to_numeric(df_raw[tcol], errors="coerce"),
        "sig2":  pd.to_numeric(df_raw[Bcol], errors="coerce"),
    }).dropna(how="any")
    return df

def _read_csv_safely(csv_path: Path) -> pd.DataFrame:
    # 忽略以 % 開頭的 Moku 註解，sep=None 讓 pandas 自動判斷逗點/Tab/分號
    df = pd.read_csv(
        csv_path, engine="python", sep=None, comment="%",
        skip_blank_lines=True, encoding="utf-8-sig"
    )
    df = df.dropna(axis=1, how="all").dropna(axis=0, how="all")
    df.columns = [c.strip() for c in df.columns]
    return df.reset_index(drop=True)


def _choose_xy(df: pd.DataFrame) -> tuple[str, str]:
    """
    挑選 x, y 欄位：
    優先使用 ('B','demod')；其次 ('B','S')；再其次 ('Ab','demod')；
    都沒有就找數值欄，動態範圍大者當 x，小者當 y（最保守 fallback）。
    """
    cols = {c.lower(): c for c in df.columns}
    # 先明確的命名
    if "b" in cols and "demod" in cols:
        return cols["b"], cols["demod"]
    if "b" in cols and "s" in cols:
        return cols["b"], cols["s"]
    if "ab" in cols and "demod" in cols:
        # 你之前的 dispersive 是把 (Ab, demod) 視為 (x,y)
        return cols["ab"], cols["demod"]

    # 退而求其次：找數值欄
    num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    if len(num_cols) < 2:
        # 全部數值化試試
        try:
            dfn = df.apply(pd.to_numeric, errors="coerce")
            num_cols = [c for c in dfn.columns if pd.api.types.is_numeric_dtype(dfn[c])]
        except Exception:
            pass
    if len(num_cols) < 2:
        raise ValueError("無法判定 x/y 欄位，請檢查 CSV 欄名或內容。")

    ranges = sorted(
        [(df[c].astype(float).max() - df[c].astype(float).min(), c) for c in num_cols],
        reverse=True
    )
    x_col = ranges[0][1]
    # 避免同欄
    y_col = next((c for _, c in ranges[1:] if c != x_col), ranges[1][1])
    return x_col, y_col



def _compute_noise(noise_df_raw: pd.DataFrame) -> float:
    """
    3–80 Hz band-mean ASD（V/√Hz），與單獨計算完全對齊：
      - lowband 路徑（可開關預處理）
      - Hann + median
      - 固定 df ~ NOISE_LOCK_DF_HZ
      - 不做 power-of-2 snap
      - detrend 依 NOISE_DETREND
    """
    df = _standardize_noise_df(noise_df_raw)

    # 鎖死所有會影響結果的參數
    kwargs_common = dict(
        band=NOISE_BAND,
        window=NOISE_WINDOW,
        average=NOISE_AVERAGE,
        min_freq_resolution=NOISE_LOCK_DF_HZ, # 直接鎖定 df
        force_long_segments=False,             # 不為了 df 犧牲段數（跟單獨算一致就好）
        detrend=NOISE_DETREND,
        snap_to_power_of_2=False               # 不調整到 2 的次方，避免 df 漂移
    )

    if NOISE_MODE == "lowband":
        _, mean_rms, _, info = noise_psd_lowband(
            df,
            enable_auto_preprocess=NOISE_ENABLE_PREPROCESS,
            # 其餘預處理參數採用函式預設（lowband_hi=150, oversample=5, fs_floor=250, fs_ceil=800, lp_margin=0.30）
            **kwargs_common
        )
    else:
        _, mean_rms, _, info = noise_psd(
            df,
            **kwargs_common
        )

    # （可選）在除錯時印出，確認實際 band 與 df 是否如預期
    # print({"band_range": info.get("band_range"), "df": info.get("df"),
    #        "n_segments": info.get("n_segments"), "preprocessed": info.get("preprocessed")})

    return float(mean_rms)  # V/√Hz

_session_re = re.compile(SESSION_NAME_PATTERN)
def _looks_like_session_name(name: str) -> bool:
    """像 01/02/03 或 01_xx/02-yy 就當作 session。"""
    return bool(re.match(r"^\d{2,}([ _-].*)?$", name))

def _progress_printer(total: int, step: int = 10):
    """
    ROOT 層進度（10%/20%…），同時顯示 已完成/總數。
    用法：upd(k, prefix='[ROOT] ')；k 從 1..total
    """
    fired = set()
    def upd(done: int, prefix: str = ""):
        if total <= 0:
            return
        pct = int(done * 100 / total)
        bucket = (pct // step) * step
        if bucket not in fired and pct >= step:
            fired.add(bucket)
            print(f"{prefix}{pct}%  ({done}/{total})")
    return upd

def _write_outputs(root_dir: Path, sheets: dict[str, pd.DataFrame], sheet_mode: str = "per_session"):
    if not sheets:
        print("[WARN] 沒有可輸出的資料。")
        return

    df_all = pd.concat(sheets.values(), ignore_index=True)

    # 在終端機列印 ALL，避免你還要開檔
    try:
        print("\n" + "="*80)
        print("ALL (彙整總表)".center(80))
        print("="*80)
        
        # 更清楚的顯示設定
        with pd.option_context(
            "display.max_rows", None,
            "display.max_columns", None, 
            "display.width", 200,           # 加寬
            "display.max_colwidth", 15,     # 限制單欄寬度
            "display.precision", 3,         # 數字精度
            "display.float_format", '{:.3g}'.format,  # 科學記號格式
            "display.colheader_justify", "center"     # 表頭置中
        ):
            # 使用 tabulate 風格的表格（如果欄位太多可以分段顯示）
            if len(df_all.columns) > 10:
                # 分段顯示：基本資訊 + 各類指標
                basic_cols = [col for col in df_all.columns if any(x in col.lower() 
                             for x in ['label', 'folder', 'current', 'power', 'range'])]
                metric_cols = [col for col in df_all.columns if col not in basic_cols]
                
                print("\n📋 基本資訊：")
                print(df_all[basic_cols].to_string(index=False))
                
                print(f"\n📊 測量結果：")
                print(df_all[metric_cols].to_string(index=False))
            else:
                print(df_all.to_string(index=False))
                
        print("="*80)
        print(f"共 {len(df_all)} 筆資料")
        print("="*80)
        
    except Exception as e:
        print(f"[WARN] 終端列印失敗：{e}")

    # CSV（ALL）
    csv_path = root_dir / f"{root_dir.name}.csv"
    df_all.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"[OK] 已輸出 CSV：{csv_path}")

    # XLSX
    xlsx_path = root_dir / f"{root_dir.name}.xlsx"
    with pd.ExcelWriter(xlsx_path, engine="openpyxl") as xw:
        if sheet_mode == "all_only":
            df_all.to_excel(xw, sheet_name="ALL", index=False)
        else:
            for name, df in sheets.items():
                sheet = re.sub(r'[\\/*?:\[\]]', '_', name)[:31] or "Sheet"
                df.to_excel(xw, sheet_name=sheet, index=False)
            if len(sheets) > 1:
                df_all.to_excel(xw, sheet_name="ALL", index=False)

    print(f"[OK] 已輸出 XLSX：{xlsx_path}")
    


def _extract_current_from_label(label: str, offset=0.0) -> float | None:
    m = re.search(r"([-+]?\d+(?:\.\d+)?)", label)
    return (float(m.group(1)) + offset) if m else None

def process_session(session_dir: Path, rules_from_txt: list, root_dir: Path) -> pd.DataFrame:
    rows = []

    # 這個 session 底下的實驗資料夾（排除 noise/）
    subdirs = [p for p in sorted(session_dir.iterdir())
               if p.is_dir() and p.name != NOISE_SUBDIR]

    # ✅ 若沒有任何子資料夾，就直接把 session_dir 自己當成一筆實驗
    if not subdirs:
        subdirs = [session_dir]

    # 進度條（10%、20%…）
    upd = _progress_printer(len(subdirs))

    for j, sub in enumerate(subdirs, start=1):
        tag = session_dir.name if session_dir != root_dir else "SINGLE"
        upd(j, prefix=f"[{tag}] ")

        # 解析掃場範圍
        B_range_used, _meta = _choose_rule_for_label(rules_from_txt, sub.name)
        if not B_range_used:
            B_range_used = B_RANGE_NT

        # 注意：參數名是 root
        scan_csv = _pick_scan_csv(sub, root=session_dir)
        if scan_csv is None:
            continue

        # 讀掃描檔
        try:
            df_raw = _read_csv_safely(scan_csv)
            df_scan = _standardize_scan_df(df_raw)
        except Exception as e:
            print(f"[WARN] {session_dir.name}/{sub.name}: 掃描檔解析失敗：{e}")
            continue

        # 切一段掃場並映射 B
        try:
            BField, AbCut, DemodCut = one_cycle_cut(
                df_scan.rename(columns={"tri":"tri"}),
                B_range=B_range_used,
                segment=LINE_SEGMENT,
                direction=LINE_DIRECTION,
                time_col="time", ab_col="Ab", demod_col="demod", tri_col="tri",
                map_mode='global', smooth_win=5
            )
            df_bs = pd.DataFrame({"B": BField, "Ab": AbCut, "demod": DemodCut})
        except Exception as e:
            print(f"[WARN] {session_dir.name}/{sub.name}: one_cycle_cut 失敗：{e}")
            df_bs = pd.DataFrame({"Ab": df_scan["Ab"], "demod": df_scan["demod"]})

        # 需求判斷
        need_slope = _want("slope") or _want("sloper2") or _want("sensitivity")
        need_noise = _want("noisepsd") or _want("sensitivity")
        need_voigt = any(_want(k) for k in ("voigtfwhm","voigtgamma","voigtsigma","voigtR2"))
        need_lor   = any(_want(k) for k in ("lorentzfwhm","lorentzR2"))
        need_gau   = any(_want(k) for k in ("gaussianfwhm","gaussianR2"))

        # ---- slope（dispersion：固定中心 + 固定窗）----
        slope_val = np.nan   # V/nT
        slope_R2  = np.nan
        disp = None
        if need_slope:
            try:
                if "B" in df_bs.columns:
                    # 只用聚焦窗資料
                    B_all = df_bs["B"].to_numpy(float)
                    Y_all = df_bs["demod"].to_numpy(float)
        
                    Bspan = float(B_all.max() - B_all.min())
                    half  = max(1e-9, Bspan * (WIDTHRATIO/100.0))
                    m = (B_all >= (DISP_CENTER - half)) & (B_all <= (DISP_CENTER + half))
                    # 點數不足就逐步放寬
                    if m.sum() < 12:
                        for s in (1.5, 2.0, 3.0):
                            mm = (B_all >= (DISP_CENTER - half*s)) & (B_all <= (DISP_CENTER + half*s))
                            if mm.sum() >= 12:
                                m = mm; break
                        else:
                            m = np.ones_like(B_all, dtype=bool)
        
                    Bf, Yf = B_all[m], Y_all[m]
                    df_focus = pd.DataFrame({"x": Bf, "y": Yf})
        
                    # 跟單獨算一致：固定中心 + amplitude 0.01V
                    disp = dispersion_lorentz_fit(
                        df_focus,
                        p0={'center': DISP_CENTER, 'amplitude': DISP_P0_AMPL},
                        max_rel_err=0.01
                    )
        
                    slope_val = float(disp["Slope"])   # a*b  (V/nT)
                    slope_R2  = float(disp["R2"])
        
                    # --- 繪圖（聚焦窗資料 + 擬合 + 線性範圍/近似）---
                    if PLOT_ENABLED and np.isfinite(slope_val):
                        try:
                            lin_lo, lin_hi = disp.get("LinearRange", (None, None))
                            ctr   = disp["Params"]["center"]
                            c0    = disp["Params"]["offset"]
                            yfit  = disp["Data"]["S_fit"]  # 與聚焦窗同長度
                            _plot_and_save_dispersion(
                                sub_dir=sub,
                                B=Bf, demod=Yf, yfit=yfit,
                                slope=slope_val, r2=slope_R2,
                                lin_range=(lin_lo, lin_hi),
                                ctr=ctr, offset=c0
                            )
                        except Exception as e:
                            print(f"[WARN] slope 繪圖呼叫失敗：{sub.name}: {e}")
        
                else:
                    # 沒 B 軸就退回 (Ab, demod)，但這種情況通常不建議算 slope
                    df_disp = df_bs.rename(columns={"Ab": "x", "demod": "y"})[["x", "y"]].dropna()
                    disp = dispersion_lorentz_fit(df_disp, p0={'center': 0.0, 'amplitude': DISP_P0_AMPL}, max_rel_err=0.01)
                    slope_val = float(disp["Slope"])
                    slope_R2  = float(disp["R2"])
            except Exception as e:
                print(f"[WARN] {sub.name}: dispersion 擬合失敗：{e}")

        # ---- 線型（Lorentz / Gaussian / Voigt）----
        lorentz_FWHM = gaussian_FWHM = voigt_FWHM = np.nan
        lorentz_R2 = gaussian_R2 = vR2 = np.nan
        v_sigma = v_gamma = np.nan

        warnings.filterwarnings(
            "ignore",
            message=r"Using UFloat objects with std_dev==0.*",
            category=UserWarning,
            module=r"uncertainties\.core"
        )

        lf = gf = vf = None
        if "B" in df_bs.columns:
            if need_lor:
                try:
                    lf = lorentz_fit(df_bs[["B","Ab"]])
                    lorentz_FWHM = float(lf["FWHM"]) if "FWHM" in lf else 2.0*abs(float(lf["gamma"]))
                    lorentz_R2   = _first_scalar(lf.get("R2"))
                except Exception as e:
                    print(f"[WARN] {sub.name}: Lorentz 擬合失敗：{e}")

            if need_gau:
                try:
                    gf = gauss_fit(df_bs[["B","Ab"]])
                    gaussian_FWHM = float(gf["FWHM"]) if "FWHM" in gf else 2.0*np.sqrt(2.0*np.log(2.0))*abs(float(gf["sigma"]))
                    gaussian_R2   = _first_scalar(gf.get("R2"))
                except Exception as e:
                    print(f"[WARN] {sub.name}: Gaussian 擬合失敗：{e}")

            if need_voigt:
                try:
                    vf      = voigt_fit(df_bs[["B","Ab"]])
                    v_sigma = float(vf.get("sigma", np.nan))
                    v_gamma = float(vf.get("gamma", np.nan))
                    vR2     = _first_scalar(vf.get("R2"))
                    L = 2.0*abs(v_gamma)
                    G = 2.0*np.sqrt(2.0*np.log(2.0))*abs(v_sigma)
                    voigt_FWHM = 0.5346*L + np.sqrt(0.2166*L*L + G*G)
                except Exception as e:
                    print(f"[WARN] {sub.name}: Voigt 擬合失敗：{e}")

            # 作圖（lineshape 疊圖）
            if PLOT_ENABLED and (need_lor or need_gau or need_voigt):
                _plot_and_save_lineshapes(
                    sub_dir=sub,
                    B=df_bs["B"].to_numpy(),
                    Ab=df_bs["Ab"].to_numpy(),
                    want_lor=need_lor, lf=(lf if need_lor else None),
                    want_gau=need_gau, gf=(gf if need_gau else None),
                    want_voi=need_voigt, vf=(vf if need_voigt else None)
                )

        # ---- 雜訊 / PSD ----
        noise_rms = np.nan
        if need_noise:
            # 用同一套挑檔邏輯：chain 版本（若你有寫）；否則直接用資料夾內 noise/
            try:
                noise_csv = _pick_noise_csv_chain(sub, session_dir, root_dir)
            except NameError:
                noise_csv = _pick_noise_csv(sub / NOISE_SUBDIR)

            if noise_csv is not None:
                try:
                    # 算均值
                    noise_rms = _compute_noise(_read_csv_safely(noise_csv))
                    # 畫 PSD
                    if PLOT_ENABLED:
                        noise_df_raw = _read_csv_safely(noise_csv)
                        noise_df_std = _standardize_noise_df(noise_df_raw)
                        _plot_and_save_psd(noise_dir=sub/NOISE_SUBDIR, noise_df_std=noise_df_std)
                except Exception as e:
                    print(f"[WARN] {session_dir.name}/{sub.name}: 噪聲計算或繪圖失敗：{e}")

        # ---- 靈敏度（T/√Hz）----
        sensitivity = np.nan
        if _want("sensitivity") and np.isfinite(noise_rms) and np.isfinite(slope_val) and slope_val != 0:
            sensitivity = noise_rms / abs(slope_val)

        # === 輸出行 ===
        row = {
            "label": sub.name,
            "folder_name": str(sub),
            SWEEP_COL_NAME: _extract_sweep_value(sub.name),  # 由你的模式函式決定：「光強電壓」或「雷射電流」
            "B_range_used_nT": f"[{B_range_used[0]:.2f}, {B_range_used[1]:.2f}]",
        }
        if _want("slope"):        row[_METRIC_COLNAMES["slope"]]       = slope_val * 1e3
        if _want("sloper2"):      row[_METRIC_COLNAMES["sloper2"]]     = slope_R2
        if _want("noisepsd"):     row[_METRIC_COLNAMES["noisepsd"]]    = noise_rms * 1e6
        if _want("sensitivity"):  row[_METRIC_COLNAMES["sensitivity"]] = sensitivity * 1e3  #  pT/√Hz
        if _want("lorentzfwhm"):  row[_METRIC_COLNAMES["lorentzfwhm"]] = lorentz_FWHM
        if _want("lorentzr2"):    row[_METRIC_COLNAMES["lorentzr2"]]   = lorentz_R2
        if _want("gaussianfwhm"): row[_METRIC_COLNAMES["gaussianfwhm"]]= gaussian_FWHM
        if _want("gaussianr2"):   row[_METRIC_COLNAMES["gaussianr2"]]  = gaussian_R2
        if _want("voigtfwhm"):    row[_METRIC_COLNAMES["voigtfwhm"]]   = voigt_FWHM
        if _want("voigtgamma"):   row[_METRIC_COLNAMES["voigtgamma"]]  = v_gamma
        if _want("voigtsigma"):   row[_METRIC_COLNAMES["voigtsigma"]]  = v_sigma
        if _want("voigtr2"):      row[_METRIC_COLNAMES["voigtr2"]]     = vR2
        rows.append(row)

    dynamic_cols = ALWAYS_COLUMNS + [_METRIC_COLNAMES[k] for k in _METRIC_COLNAMES if k in _SELECT]
    return pd.DataFrame(rows, columns=dynamic_cols)
   
def process_root(root_dir: Path) -> pd.DataFrame:
    root_dir = root_dir.resolve()

    warnings.filterwarnings(
        "ignore",
        message=r"Using UFloat objects with std_dev==0.*",
        category=UserWarning,
        module=r"uncertainties\.core"
    )

    rules_from_txt = _load_experiment_rules(root_dir)

    # ★ 由層級自動判定
    layout, sessions, sheet_mode = _detect_layout(root_dir)

    total = len(sessions)
    upd_root = _progress_printer(total)
    sheets: dict[str, pd.DataFrame] = {}

    for i, sess in enumerate(sessions, start=1):
        tag = (sess.name if sess != root_dir else root_dir.name)
        print(f"[ROOT] 開始處理：{tag}  ({i-1}/{total})")
        df_sess = process_session(sess, rules_from_txt, root_dir)
        if len(df_sess):
            sheets[tag] = df_sess
        print(f"[ROOT] 完成：{tag}       ({i}/{total})")
        upd_root(i, prefix="[ROOT] ")

    _write_outputs(root_dir, sheets, sheet_mode=sheet_mode)

    return (pd.concat(sheets.values(), ignore_index=True) if sheets else pd.DataFrame())


root = Path(r"C:\Users\YiHsuanChen\OneDrive\桌面\Research\code\OPM_code\250925\固定光強改波長")
process_root(root)
notify_done()

[ROOT] 開始處理：01  (0/3)
[01] 10%  (1/10)
[01] 20%  (2/10)
[01] 30%  (3/10)
[01] 40%  (4/10)
[01] 50%  (5/10)
[01] 60%  (6/10)
[01] 70%  (7/10)
[01] 80%  (8/10)
[01] 90%  (9/10)
[01] 100%  (10/10)
[ROOT] 完成：01       (1/3)
[ROOT] 33%  (1/3)
[ROOT] 開始處理：02  (1/3)
[02] 10%  (1/10)
[02] 20%  (2/10)
[02] 30%  (3/10)
[02] 40%  (4/10)
[02] 50%  (5/10)
[02] 60%  (6/10)
[02] 70%  (7/10)
[02] 80%  (8/10)
[02] 90%  (9/10)
[02] 100%  (10/10)
[ROOT] 完成：02       (2/3)
[ROOT] 66%  (2/3)
[ROOT] 開始處理：03  (2/3)
[03] 10%  (1/10)
[03] 20%  (2/10)
[03] 30%  (3/10)
[03] 40%  (4/10)
[03] 50%  (5/10)
[03] 60%  (6/10)
[03] 70%  (7/10)
[03] 80%  (8/10)
[03] 90%  (9/10)
[03] 100%  (10/10)
[ROOT] 完成：03       (3/3)
[ROOT] 100%  (3/3)

                                   ALL (彙整總表)                                   

📋 基本資訊：
label                                  folder_name                                   current_mA B_range_used_nT
 128  C:\Users\YiHsuanChen\OneDrive\桌面\Research\code\OPM_code\250925\固定光強改波長\01\128  